In [1]:
import sqlite3
import datetime as dt   
import time
import csv
import requests
import pandas as pd, numpy as np
import pprint
import matplotlib.pyplot as plt
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/GGV/anaconda/envs/GGV_DASH/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
# helper functions

def sqlite_test():
	try:
		conn = sqlite3.connect('/Users/GGV/Desktop/mcdonald_listings.sqlite')
		c = conn.cursor()
		query = "select * from trafi limit 10 ;"
		df = pd.read_sql_query(query, conn)
		print (df)
		print ("Sqlite connect OK")
	except:
		print ("Sqlite connect failed")




def sqlite_query_df(query):
	try:
		conn = sqlite3.connect('/Users/GGV/Desktop/mcdonald_listings.sqlite')
		c = conn.cursor()
		df = pd.read_sql_query(query, conn)
		print ("Sqlite connect OK")
		return df 
	except:
		print ("Sqlite connect failed")

In [3]:
query= '''

select * 
FROM listings 
'''
listings = sqlite_query_df(query)

Sqlite connect OK


In [4]:
query= '''

select * 
FROM reviews 
'''
reviews = sqlite_query_df(query)

Sqlite connect OK


In [5]:
reviews.head(2)

,username,negative,neutral,positive
0,fionasohyuru,0,0,18
1,purple0603,0,0,9


In [6]:
listings.head(2)

,username,description,title,can_mail,price,meetup_details,can_meetup,likes,condition,meetup_location
0,fionasohyuru,Curly fries / Twister Fries (Taste exactly lik...,Twister fries (2.5kg),None,18.0,None,Meet-Up,3,New,None
1,purple0603,Brand new in original macdonald packaging\r\rC...,Hello Kitty - Macdonald Royal Series,None,50.0,None,None,3,None,None


#### 1. What is the average number of likes across all listings?

In [7]:
query_avg_like= '''

SELECT avg(likes)
FROM listings

'''
avg_like = sqlite_query_df(query_avg_like)
avg_like

Sqlite connect OK


,avg(likes)
0,13.789954


####  2. What is the average number of likes for used listings?

In [8]:
query_avg_like_used= '''

SELECT avg(likes)
FROM listings
WHERE condition == 'Used'
'''
avg_like_used = sqlite_query_df(query_avg_like_used)
avg_like_used

Sqlite connect OK


,avg(likes)
0,5.418033


#### 3. What is the average price for all the listings with mcdonald keyword in the title? (Hint: include all variations of mcdonalds)
 

In [9]:
query_avg_price_m_keyword= '''

SELECT avg(price)
FROM listings
WHERE LOWER(title) LIKE '%%mcdonalds%%'

'''
avg_price_m_keyword = sqlite_query_df(query_avg_price_m_keyword)
avg_price_m_keyword

Sqlite connect OK


,avg(price)
0,24.26087


#### 4. Find the number of listings unrelated to Mcdonalds.
 

In [18]:
query_num_unrelated_m= '''


WITH LIST AS
  ( SELECT *
   FROM listings
   WHERE LOWER(title) NOT LIKE '%%mcdonald%%'
     AND LOWER(description) NOT LIKE '%%mcdonald%%' )
SELECT COUNT(*)
FROM LIST


'''
num_unrelated_m = sqlite_query_df(query_num_unrelated_m)
num_unrelated_m

Sqlite connect OK


,COUNT(*)
0,2249


DONT know if "MacDonald" is related to "Mcdonalds"  :)

#### 5. List the usernames of the top 3 sellers with the highest number of positive ratings.
 

In [11]:
query_top_3_seller= '''

SELECT *
FROM reviews
ORDER BY positive DESC LIMIT 3

'''
top_3_seller = sqlite_query_df(query_top_3_seller)
top_3_seller

Sqlite connect OK


,username,negative,neutral,positive
0,bxs,0,0,1229
1,hardrockseller99,1,0,920
2,hellomisterneospacian,3,2,760


#### 6.  What is the total number of sellers with more than 200 reviews?


In [12]:
query_review= '''
SELECT count(*)
FROM
  (SELECT username,
          sum(negative) + sum(positive) + sum(neutral) AS sum_
   FROM reviews
   GROUP BY 1
   HAVING sum_ > 200
   ORDER BY sum_ DESC) sub

'''
review = sqlite_query_df(query_review)
review

Sqlite connect OK


,count(*)
0,66


#### 7. What is the total number of users with only positive reviews?


In [13]:
query_only_positive_review= '''

SELECT count(*)
FROM
  (SELECT username,
          sum(negative),
          sum(positive),
          sum(neutral)
   FROM reviews
   GROUP BY 1
   HAVING sum(negative) = 0
   AND sum(neutral) = 0
   AND sum(positive) > 0 ) sub

'''
positive_review = sqlite_query_df(query_only_positive_review)
positive_review

Sqlite connect OK


,count(*)
0,1231


#### 8.  What is the total number of listings by users with only positive reviews?


In [14]:
query_list_positive_review= '''

WITH user_list AS
  (SELECT username
   FROM reviews
   GROUP BY 1
   HAVING sum(negative) = 0
   AND sum(neutral) = 0
   AND sum(positive) > 0)
SELECT count(*)
FROM listings
JOIN user_list ON listings.username = user_list.username
ORDER BY listings.username
  

'''
list_positive_review = sqlite_query_df(query_list_positive_review)
list_positive_review

Sqlite connect OK


,count(*)
0,1890


#### 9.  For the top 3 sellers with the highest number of listings, list their usernames, total listings and the total value of their listings


In [15]:
query_top_user_detail= '''

SELECT username,
       count(username),
       sum(price),
       sum(likes)
FROM listings
GROUP BY username
ORDER BY count(username) DESC LIMIT 3

'''
top_user_detail = sqlite_query_df(query_top_user_detail)
top_user_detail

Sqlite connect OK


,username,count(username),sum(price),sum(likes)
0,eliset08,62,2115.0,344
1,tmatma,56,695.0,172
2,scotty2198,24,663.0,3694


#### 10.  For the top 3 sellers with the most reviews, list their usernames, total listings, and total reviews in descending number of reviews received by each seller


In [16]:
query_top_user_list_review = '''

WITH top_user AS
  (SELECT username,
          sum(negative) + sum(positive) + sum(neutral) AS sum_
   FROM reviews
   GROUP BY 1
   ORDER BY sum_ DESC LIMIT 3),
     listing AS
  ( SELECT top_user.username AS username,
           count(listings.username) AS listing_
   FROM listings
   JOIN top_user ON top_user.username = listings.username
   GROUP BY 1 )
SELECT top_user.*,
       listing.listing_
FROM listing
JOIN top_user ON top_user.username = listing.username

'''
top_user_list_review = sqlite_query_df(query_top_user_list_review)
top_user_list_review

Sqlite connect OK


,username,sum_,listing_
0,bxs,1229,1
1,hardrockseller99,921,1
2,hellomisterneospacian,765,1
